# Traffic Sign Recognition Model
## Environment Initialisation

In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("My Drive/[ Masters ] - Deep Learning Proj/Model")
!pip install import_ipynb

In [2]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [3]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


## Reading DataSet
Pre-processing and Loading Train and Test DataSet respectively

In [ ]:
def ReadTrainDataSet():
    print("Reading Train Pre-processed DataSet")
    processedTrainDataSet = 'Processed_DataSet/BelgiumTrainDataSet.pkl'
    isProcessedTrainDataSetExits= os.path.exists(processedTrainDataSet)
    train_image_array, train_image_labels = None, None
    
    if isProcessedTrainDataSetExits:
        print("Loading Processed Train DataSet from Processed_DataSet/BelgiumTrainDataSet.pkl")
        file = open(processedTrainDataSet, 'rb')
        train_image_array, train_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Train DataSet.")
        
    else:
        print("Processed_DataSet/BelgiumTrainDataSet.pkl file does not exist")
        print("Loading Train DataSet ... This may take a while.")
        train_image_array, train_image_labels =  LoadTrainDataSetWithAugmentation()
        file = open(processedTrainDataSet, 'wb')
        pickle.dump((train_image_array, train_image_labels), file, protocol=4)
        file.close()
        print("Saving pre-processed train DataSets in Processed_DataSet/BelgiumTrainDataSet.pkl")
    return train_image_array, train_image_labels    

In [9]:
def ReadTestDataSet():
    processedTestDataSet = 'Processed_DataSet/BelgiumTestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/BelgiumTestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/BelgiumTestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/BelgiumTestDataSet.pkl")
    return test_image_array, test_image_labels

In [ ]:
train_image_array, train_image_labels= ReadTrainDataSet()

In [ ]:
print(train_image_array.shape)

In [10]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/BelgiumTestDataSet.pkl
Done Loading Test DataSet.


In [11]:
print(test_image_array.shape)

(2520, 60, 60, 1)


## Train-Split 
Split Train images dataset into two splits: training and validation respectively 80-20

In [ ]:
image_train, image_valid,label_train, label_valid = train_test_split(train_image_array, train_image_labels,stratify=train_image_labels,test_size=0.1,random_state=43)

In [ ]:
print(image_train.shape)

In [ ]:
del train_image_array
del train_image_labels

## Hyper-parameters Tuning

In [ ]:
# Hyper-parameters tuning
kernel_2 = (3,3)
pooling = (2,2)
dropout = 0.2
num_classes = 62

## Model Architecture

In [ ]:
import tensorflow as tf
import keras

from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, Add,AveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, SeparableConv2D,BatchNormalization,Dropout,MaxPool2D,Flatten,Dense
from keras.layers import concatenate

from keras import initializers
from keras import backend as K
from keras.layers import Input
from keras.models import Model

In [ ]:
weight_decay=1E-4
from keras.regularizers import l2

def model_():
    model = None
    tf.initializers.Orthogonal(gain=1.0, seed=None)
    model=Sequential()

    model.add(Conv2D(32,kernel_2, input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(128,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(256,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation = tf.nn.relu))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation = 'softmax'))
    return model

In [ ]:
from tensorflow import keras
model = model_()

## Model Optimizer 

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2,  verbose=1, min_delta=1e-4, min_lr=1e-20)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

In [ ]:
x=aug.flow(image_train, np.array(label_train), batch_size=32)

## Model Training

In [ ]:
history = model.fit(x, epochs=94, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

## Model Evaluation

In [12]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

2520/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3091713990957018, 0.98968256]

## Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Model Summary 
Number of parameters used in the model architecture

In [ ]:
print(model.summary())

## Saving Trained .h5 Model

In [ ]:
def saveCNNModel(model_name):
    model.save(model_name)

In [ ]:
saveCNNModel('Trained_Models/Belgium_Winning_98_97.h5')

In [ ]:
model._set_inputs(image_train,np.array(label_train))

## Loading Saved Trained .h5 Model

In [ ]:
del model

In [4]:
def load_model():
    Model = tf.keras.models.load_model('Trained_Models/Belgium_Winning_98_97.h5')
    return Model

In [5]:
model= load_model()

In [7]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

## Computation Avg. Processing Time

In [ ]:
import time
t1=time.time()
model.predict(test_image_array)
t2=time.time()
print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")